Artifical Start

In [76]:
using LinearAlgebra
using SparseArrays
using LightGraphs
using SimpleWeightedGraphs
using GraphPlot
using BenchmarkTools
using Random
include("minFlow.jl")

AddToTable!

In [94]:
nodes = 5
E = [(1,2) (1,3) (1,5) (2,3) (3,4) (4,2) (4,5) (5,3)]
cost_dict = Dict((1,2)=> 1,(1,3)=> 2, (1,5)=> 1, (2,3)=> 2, (3,4)=> 1, (4,2)=> 1, (4,5)=> 1, (5,3)=> 1)

## Artifical Start
n = nodes + 1 
b = Int64[2;5;1;-4;-4;0] # Flow
c = Int64[1;1;1;1;1;0] # Artifical Cost
x = abs.(b)

6-element Vector{Int64}:
 2
 5
 1
 4
 4
 0

In [100]:
nodes = 8
E = [(1,2) (1,3) (1,4) (2,6) (2,7) (3,5) (3,6) (4,5) (6,5) (6,7) (8,3)]
cost_dict = Dict((1,2)=>2, (1,3)=>-9, (1,4)=>2, (2,6)=>-2, (2,7)=>2, (3,5)=>2, (3,6)=>2, (4,5)=>2, (6,5)=>2, (6,7)=>2, (8,3)=>2)

## Artifical Start
n = nodes + 1 
b = Int64[2;2;-2;-2;2;-2;-2;2;0] # Flow
c = Int64[1;1;1;1;1;1;1;1;0] # Artifical Cost
x = abs.(b)

9-element Vector{Int64}:
 2
 2
 2
 2
 2
 2
 2
 2
 0

In [101]:
B = ArtificalBase_table(b)
B_edgeList = ArtificalBase_list(b)
B_matrix = BaseMatrix(B_edgeList,n)
NB_edgeList = copy(E)


1×11 Matrix{Tuple{Int64, Int64}}:
 (1, 2)  (1, 3)  (1, 4)  (2, 6)  (2, 7)  …  (4, 5)  (6, 5)  (6, 7)  (8, 3)

In [102]:
w = zeros(n,1)
valid = true
while (sum(c) != 0)
    w = c'/B_matrix    
    max ,enteringEdge = ReducedCost(w,NB_edgeList,1, 0)
    
    println("$max , $enteringEdge")
    if(max == 0)
        println("ENDING!!")
        break
    end
    edge_u = enteringEdge[1]
    edge_v = enteringEdge[2]
    path = Find_path(edge_u,edge_v,n,B)
    #println("($edge_u ,$edge_v)", path)
    Flow_list, valid = Get_flow(path)
    println(Flow_list, valid)
    if valid
        indexes = Get_Indexes(path,B_edgeList) 
        delta, exitingEdge, min_index  = getDeleta(Flow_list, path, indexes, x)
        updateFlowPattern!(Flow_list, path, indexes, min_index, delta, x)
        edge_i = exitingEdge[1]
        edge_j = exitingEdge[2]
        RemoveFromTable!(edge_i,edge_j,n,B)
        RemoveFromTable!(edge_j,edge_i,n,B)
        AddToTable!(edge_u, edge_v,1,B)
        AddToTable!(edge_v, edge_u,-1,B)
        Update_BaseMatrix!(B_matrix,B_edgeList,exitingEdge,enteringEdge)
        c[min_index] = 0
    end
    for (i,e) in enumerate(NB_edgeList) 
        if e == enteringEdge
            NB_edgeList[i] = (0,0)
            break
        end
    end
end

2.0 , (1, 3)
[1, -1, -1]true
2.0 , (1, 4)
[1, 1, 1]false
2.0 , (2, 6)
[1, -1, -1]true
2.0 , (2, 7)
[1, 1, 1]false
2.0 , (6, 7)
[1, 1, -1, -1]true
2.0 , (1, 2)
[1, 1, 1, -1, -1]true
0.0 , (3, 5)
ENDING!!


In [105]:
B_edgeList

9×1 Matrix{Tuple{Int64, Int64}}:
 (1, 9)
 (1, 2)
 (1, 3)
 (9, 4)
 (5, 9)
 (2, 6)
 (9, 7)
 (8, 9)
 (9, 9)

In [103]:
NB_edgeList

1×11 Matrix{Tuple{Int64, Int64}}:
 (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)  …  (4, 5)  (6, 5)  (0, 0)  (8, 3)

In [104]:
basicEdge =  Base_list(B_edgeList, nodes)
cost = Set_cost(basicEdge, cost_dict)
NB_edgeList = NonBasic_list(basicEdge, E)
B_matrix = BaseMatrix(basicEdge,nodes)
B = Base_table(basicEdge, nodes)

b = b[1:end-1] # Flow
x = x[1:end-1]

(1, 4)
(2, 7)
(3, 5)
(3, 6)
(4, 5)


LoadError: BoundsError: attempt to access 4×1 Matrix{Tuple{Int64, Int64}} at index [5]

In [21]:
w = cost'/B_matrix
while sum((w*B_matrix)' .>= cost) != nodes
    
    max ,enteringEdge = ReducedCost(w,NB_edgeList, 2, cost_dict)
    if(max == 0)
        println("ENDING!!")
        break
    end
    edge_u = enteringEdge[1]
    edge_v = enteringEdge[2]
    path = Find_path(edge_u,edge_v,nodes,B)
    Flow_list, valid = getFlow(path,nodes)
    if valid
        indexes = Get_Indexes(path,B_edgeList) 
        delta, exitingEdge, min_index  = getDeleta(Flow_list, path, indexes, x)
        updateFlowPattern!(Flow_list, path, indexes, min_index, delta, x)
        edge_i = exitingEdge[1]
        edge_j = exitingEdge[2]
        RemoveFromTable!(edge_i,edge_j,nodes,B)
        RemoveFromTable!(edge_j,edge_i,nodes,B)
        AddToTable!(edge_u, edge_v,1,B)
        AddToTable!(edge_v, edge_u,-1,B)
        updateBaseMatrix!(B_matrix,B_edgeList,exitingEdge,enteringEdge)
        cost[min_index] = cost_dict[enteringEdge]
    end
    w = cost'/B_matrix
end

In [22]:
#Check Feasibility
(w*B_matrix)' .>= cost

5×1 BitMatrix:
 1
 1
 1
 1
 1

In [23]:
x

5-element Vector{Int64}:
 2
 0
 6
 5
 2

In [24]:
cost

5×1 Matrix{Float64}:
  1.0
  0.0
  1.0
 10.0
  1.0

In [25]:
x

5-element Vector{Int64}:
 2
 0
 6
 5
 2

In [26]:
#Min Cost
cost'x

1-element Vector{Float64}:
 60.0